In [43]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import folium

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 5.16 ms (started: 2021-12-03 16:42:38 -05:00)


<h1 align="center">An Analysis of Worldwide Air Quality</h1>

## Problem Statement

Our goal of the project was the answer the following question: 

- Is there a correlation between population size and air quality?
- Additionally, is there a correlation between elevation and air quality?
- Finally, how does each of the three pollutants correlate with air quality?

## Data

To answer our questions, we used data from two APIs: Air Quality Programmatic API and the Spott REST API. 

The Air Quality Programmatic API provides city level data about air quality index and individual pollutants (PM2.5, PM10, Ozone (O3)). The API also provides data on city latitude and longitude. 

The Spott API provides city level data on population and elevation through directly searching for the city's name. 

### Air Quality Programmatic API

First, we webscraped data, using Beautiful Soup, to gather the 1000 most populated cities in the world. The website we scraped data from is https://data.mongabay.com/cities_pop_01.htm'.  

Once we webscrapped the 1000 most populated cities, we ran the list of cities through the Air Quality Programmatic API. After dropping N/A data, we stored the data in a csv format. There are 526 rows and 9 columns. The size of the stored data is 35 KB.

In [28]:
air_quality_api = pd.read_csv('waqi_df.csv')
air_quality_api.sample(5)

,Unnamed: 0,cities,aqi,latitude,longitude,pm25,pm10,o3,uvi,date
85,114,Rome,30,41.893267,12.475385,30.0,16.0,11.0,0.0,11-30-2021
321,552,Jinzhou,78,41.095120,121.127004,78.0,17.0,9.0,0.0,11-30-2021
19,23,Nagoya,26,35.181446,136.906398,17.0,18.0,2.0,1.0,11-30-2021
213,329,Yiyang,95,28.553860,112.355180,95.0,28.0,10.0,1.0,11-30-2021
105,144,Tampa,19,27.928333,-82.454444,19.0,11.0,10.0,1.0,11-30-2021


time: 19.1 ms (started: 2021-12-03 16:22:50 -05:00)


### Spott API

For the Spott API, are main contraint was the number of free requests we received. Per account, Spott only granted us 1000 requests. To limit the number of requests, we made a list of cities from the Air Quality API and ran those cities through the Spott API.

We stored the data in a csv format. There are 526 rows and 3 columns. The size of the stored data is 14 KB.

In [29]:
spott_api = pd.read_csv('spott_df.csv')
spott_api.sample(5)

,Unnamed: 0,Cities,Population,Elevation
217,217,Huaian,NaN,NaN
368,368,Nantong,666251.0,6.0
59,59,Barcelona,1621537.0,15.0
104,104,Nanchang,2357839.0,20.0
40,40,Santiago,4837295.0,556.0


time: 16 ms (started: 2021-12-03 16:22:52 -05:00)


## Data Cleaning and Manipulation - Julia

## Time Complexity - Together

## Analysis - Analese

## Take-aways

The factor that by far had the highest correlation with Air Quality Index was the measure of the PM25 pollutant. The relationship is strong and appears to be significant. Further testing will more thoroughly analyze the significance of the relationship between PM25 levels and Air Quality Index levels. Another route we could take in the future is to gather data from cities other than the 1000 most populated cities in the world. We wonder how the correlations we analyzed would differ from medium-sized cities and small towns.

## References